In [ ]:
from grid.interface import *

In [ ]:
boardsize = 9
limits = {3: 8, 5: 15, 7: 45, 9: 90}
desc = f'main/{boardsize}'
acks = acknowledged(desc)
for width in [64, 128, 256, 512, 1024]:
    for depth in [1, 2, 4, 8, 16, 32]:
        params = dict(width=width, depth=depth, boardsize=boardsize, timelimit=limits[boardsize]*60, desc=desc)
        if is_missing(params, acks):
            log.info(f'Launching {params}')
            jittens.jobs.submit(
                cmd='python -c "from boardlaw.main import *; run_jittens()" >logs.txt 2>&1',
                dir='.',
                resources={'gpu': 1},
                params=params)


In [ ]:
import time
from IPython import display

vast.jittenate(local=True, ssh_accept=True)
last_fetch = 0
while not jittens.finished():
    try:
        display.clear_output(wait=True)
        jittens.refresh()
        time.sleep(15)

        if time.time() > last_fetch + 600:
            fetched = fetch()
            jittens.manage.cleanup(fetched)
            last_fetch = time.time()
    except Exception as e:
        log.info(f'Failed with error {e}')
        time.sleep(60)

fetched = fetch()
jittens.manage.cleanup(fetched)